In [11]:
import torch
from models.ViTPose.vitpose import get_vitpose_model
import cv2
from utils import get_keypoints_from_heatmaps

In [24]:
model = get_vitpose_model("B",17,"simple")

In [25]:
weights = torch.load("./weights/vitpose-b-simple.pth")['state_dict']
model.init_weights(weights)

Missing keys:  ['backbone.cls_token']


In [21]:
img_path = "./example_images/demo_cropped.jpg"
img = cv2.imread(img_path)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (192, 256))
img = img / 255.0
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
img = (img - mean) / std
img = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float()

print(img.shape)

torch.Size([1, 3, 256, 192])


In [22]:
model.eval()

with torch.no_grad():
    out = model(img)
    print(out.shape)

torch.Size([1, 17, 64, 48])


In [23]:
image_show = cv2.imread(img_path)
image_show = cv2.resize(image_show, (192, 256))
# heatmaps = out.permute(0, 1, 3, 2)
heatmaps = out

# get keypoints from (17, 64, 48) heatmap
keypoints = get_keypoints_from_heatmaps(heatmaps, (256, 192))[0]

# # sum all the heatmaps
# heatmap = heatmaps.sum(axis=0).cpu().numpy()

# # normalize the heatmap
# heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

# heatmap = cv2.resize(heatmap, (192, 256))
# heatmap = cv2.applyColorMap((heatmap * 255).astype('uint8'), cv2.COLORMAP_JET)

for i, keypoint in enumerate(keypoints):
    x, y = keypoint
    cv2.circle(image_show, (int(x), int(y)), 3, (0, 255, 0), -1)

# cv2.imshow("heatmap", heatmap)
cv2.imshow("keypoints", image_show)
cv2.waitKey(0)
cv2.destroyAllWindows()